In [1]:
import torch
import torchvision

In [3]:
class MyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(784, 128)
        self.fc2 = torch.nn.Linear(128, 10)

    def forward(self, x):
        x = x.reshape((x.shape[0], 784)) # input shape: 32*28*28 , output shape: 32*784
        x = self.fc1(x)
        x = torch.relu(x)
        x = torch.dropout(x, 0.2, train=True)
        x = self.fc2(x)
        x = torch.softmax(x, dim=1)
        return x

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MyModel()
model = model.to(device)

In [5]:
# Hyper Parameters
batch_size = 64
epochs = 10
lr = 0.01

In [ ]:
# Dataset
transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0), (1))
])

dataset = torchvision.datasets.MNIST("./dataset", train=True, download=True, transform=transform)
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_function = torch.nn.CrossEntropyLoss()

In [2]:
def calc_acc(preds, labels):
    preds_max = torch.argmax(preds, 1)
    acc = torch.sum(preds_max == labels.data, dtype=torch.float64) / len(preds)
    return acc

In [8]:
# train
model.train(True)

for epoch in range(epochs):
    train_loss = 0.0
    train_acc = 0.0
    for images, labels in train_data_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        # 1- forwarding
        preds = model(images)
        # 2- backwarding 
        loss = loss_function(preds, labels)
        loss.backward()
        # 3- Update
        optimizer.step()

        train_loss += loss
        train_acc += calc_acc(preds, labels)
    
    total_loss = train_loss / len(train_data_loader)
    total_acc = train_acc / len(train_data_loader)

    print(f"Epoch: {epoch}, Loss: {total_loss}, Acc: {total_acc}")

torch.save(model.state_dict(), "mnist.pth")

Epoch: 0, Loss: 1.5730654001235962, Acc: 0.8901419243070362
Epoch: 1, Loss: 1.5410693883895874, Acc: 0.9200093283582089
Epoch: 2, Loss: 1.5372532606124878, Acc: 0.9236240671641791
Epoch: 3, Loss: 1.5351698398590088, Acc: 0.9258562100213219
Epoch: 4, Loss: 1.5321077108383179, Acc: 0.9289378997867803
Epoch: 5, Loss: 1.5306951999664307, Acc: 0.9301039445628998
Epoch: 6, Loss: 1.5291517972946167, Acc: 0.9319529584221748
Epoch: 7, Loss: 1.528383493423462, Acc: 0.9325859541577824
Epoch: 8, Loss: 1.5266255140304565, Acc: 0.9344016524520256
Epoch: 9, Loss: 1.527269721031189, Acc: 0.9337853144989339


In [ ]:
# inference
import cv2
import numpy as np


model.eval()

# preprocess
img = cv2.imread("three.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (28, 28))
tensor = transform(img).unsqueeze(0).to(device)

# process
preds = model(tensor)

# postprocess
preds = preds.cpu().detach().numpy()
output = np.argmax(preds)
print(output)

3